In [1]:
### import libs
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding,  Dense, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [2]:
TRAINING_SIZE = 20000
EMBEDDING_DIM = 50
VOCAB_SIZE = 12000
NUM_EPOCHS = 30
MAX_LEN = 1000

In [3]:
data = pd.read_csv("formated_reviews.csv")

In [4]:
reviwes = data["review"]
labels = data["score"]




In [5]:
traning_data = reviwes[0:TRAINING_SIZE]
traning_labels = labels[0:TRAINING_SIZE]

testing_data = reviwes[TRAINING_SIZE:]
testing_labels = labels[TRAINING_SIZE:]


In [6]:
training_df = pd.DataFrame({"review" : traning_data, "score" : traning_labels})

training_df.head()

training_df['score'].value_counts(normalize=True) * 100

score
0    73.395
1    26.605
Name: proportion, dtype: float64

In [7]:
tokenizer  = Tokenizer(num_words=VOCAB_SIZE , oov_token="<OOV>")
tokenizer.fit_on_texts(traning_data)

training_sequences = tokenizer.texts_to_sequences(traning_data)
training_padded = pad_sequences(training_sequences, maxlen=MAX_LEN, padding="post", truncating="post")

testing_sequences = tokenizer.texts_to_sequences(testing_data)
testing_padded = pad_sequences(testing_sequences, maxlen=MAX_LEN, padding="post", truncating="post")

word_index = tokenizer.word_index

print("Found %s unique tokens." % len(word_index))

Found 10925 unique tokens.


In [8]:
print(training_padded.shape[1])

1000


In [9]:
model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=training_padded.shape[1]),
    GlobalAveragePooling1D(),
    Dense(1000, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

c:\Users\vmmdr\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model.fit(training_padded, traning_labels, epochs=NUM_EPOCHS, validation_data=(testing_padded, testing_labels))

Epoch 1/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - accuracy: 0.7368 - loss: 0.5817 - val_accuracy: 0.7345 - val_loss: 0.5863
Epoch 2/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.7418 - loss: 0.5705 - val_accuracy: 0.7345 - val_loss: 0.5791
Epoch 3/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.7284 - loss: 0.5805 - val_accuracy: 0.7345 - val_loss: 0.5738
Epoch 4/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.7318 - loss: 0.5631 - val_accuracy: 0.8078 - val_loss: 0.4519
Epoch 5/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - accuracy: 0.7715 - loss: 0.4947 - val_accuracy: 0.7517 - val_loss: 0.5371
Epoch 6/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.7986 - loss: 0.4481 - val_accuracy: 0.8038 - val_loss: 0.4159
Epoch 7/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.8139 - loss: 0.4220 - val_accuracy: 0.8002 - val_loss: 0.3951
Epoch 8/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.8213 - loss: 0.4010 - 

KeyboardInterrupt: 

In [82]:
### Testing 


sentence = ["Essa menina é muito bonita", "O teclado é lento"  , "Eu não gosto muito de facas"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=MAX_LEN, padding="post", truncating="post")
print(model.predict(padded))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[[0.07712726]
 [0.15285975]
 [0.45239732]]
